# 数据规整化：清理、转换、合并、重塑

数据**分析和建模**方面的大量编程工作都是用在数据准备上的:**加载**、**清理**、**转换**以及**重塑**。有时候,存放在文件或数据库中的数据并不能满足你的数据处理应用的要求。许多人都选择使用通用编程语言(如Python、Perl、R或Java)或UNIX文本处理工具(如sed或awk)对数据格式进行专门处理。幸运的是,pandas和Python标准库提供了一组**高级**的、**灵活**的、**高效**的**核心函数**和**算法**,它们使你能够轻松地将数据规整化为正确的形式。

In [1]:
import pandas as pd
from pandas import Series
from pandas import DataFrame

import numpy as np

## 合并数据集

pandas对象中的数据可以通过一些**内置**的方式进行合并：
* pandas.merge可根据**一个或多个键**将不同DataFrame中的**行**连接起来，实现的就是数据库的连接操作 -- **横向合并**；
* pandas.concat可以沿着一条**轴**将多个对象**堆叠**到一起 -- **纵向合并**；
* 实例方法combine_first可以将重复数据编接在一起,用一个对象中的值**填充**另一个对象中的**缺失值** -- **填充式**；

### 数据库风格的DataFrame合并 -- pandas.merge

#### 普通多对一默认列默认合并方式合并

In [2]:
df1 = DataFrame({'key':['a','b','c','a','b','c'],
                'data1':range(6)})
df2 = DataFrame({'key':['a','d'],
                'data2':range(2)})
pd.merge(df1,df2)

,data1,key,data2
0,0,a,0
1,3,a,0


#### 索引上的合并 -- left_index、right_index

简单索引情况下：

In [8]:
df1 = DataFrame({'key':['a','b','c','a','a'],
                'data':range(5)})
df2 = DataFrame({'data2':range(3)},index=['a','b','a'])
pd.merge(df1, df2, left_on='key', right_index=True, how='outer') # 需要同时指定右侧使用索引作为连接键，同时指定左侧的连接键列名

,data,key,data2
0,0,a,0.0
0,0,a,2.0
3,3,a,0.0
3,3,a,2.0
4,4,a,0.0
4,4,a,2.0
1,1,b,1.0
2,2,c,NaN


两侧都是索引的情况下：

In [14]:
df1 = DataFrame(np.arange(4), index=['a','b','a','c'])
df2 = DataFrame(np.arange(4), index=['a','b','b','d'])
pd.merge(df1, df2, left_index=True, right_index=True, suffixes=['_df1','_df2'])

,0_df1,0_df2
a,0,0
a,2,0
b,1,1
b,1,2


层次化索引情况下（此时需要显示指明用于连接索引的多个列）：

In [10]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                   'key2': [2000, 2001, 2002, 2001, 2002],
                   'data': np.arange(5.)})
righth = DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])
# 其中righth是一个层次化索引的DataFrame，直接指定right_index=True试试看，但是此时需要指定left_on使用多个键来连接
pd.merge(lefth, righth, left_on=['key1','key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


#### merge函数参数描述

* left：用于合并的左侧DataFrame；
* right：用于合并的右侧DataFrame；
* how：合并方式，“inner”（保留交集）、“outer”（保留并集）、“left”（保留左侧）、“right”（保留右侧），默认为inner；
* on：用于连接的列名，必须同时存在于左右两侧，如果本身以及left_on、right_on都未指定，则使用左右两侧的列名交集来连接；
* left_on：左侧DataFrame中用于连接的列名；
* right_on：右侧DataFrame中用于连接的列名；
* left_index：将左侧行索引用作其连接键，设置为True表示左侧索引也可以作为连接键；
* right_index：将右侧行索引用作其连接键，设置为True表示右侧索引也可以作为连接键；
* sort：根据连接的键对数据进行排序，默认是True，但是在处理大数据时，设置为False可以得到更好的性能；
* suffixes：字符串值元组，用于追加到重叠的列名后，默认是“_x”、“_y”，如果有列data重复，那么合并后会得到data_x、data_y；
* copy：是否复制到结果数据结构中，默认为True；

**警告**: 在进行列-列连接时,DataFrame对象中的索引会被丢弃；

### DataFrame.join方法实现索引上的合并

#### 左右两侧均使用索引进行连接

In [22]:
df1 = DataFrame(np.arange(4), index=['a','b','a','c'])
df2 = DataFrame(np.arange(4), index=['a','b','b','d'])
# 默认how是left，使用两侧的索引来连接，在有重复列名时必须指定lsuffix以及rsuffix
df1.join(df2, lsuffix='_left', rsuffix='_right', how='left') 

,0_left,0_right
a,0,0.0
a,2,0.0
b,1,1.0
b,1,2.0
c,3,NaN


#### 左侧使用列作为索引进行连接

In [27]:
df1 = DataFrame({'key':['a','b','c','a','a'],
                'data':range(5)})
df2 = DataFrame({'data2':range(3)},index=['a','b','a'])
# 可以指定左侧的列作为索引连接右侧的索引
df1.join(df2, on='key', how='inner')

,data,key,data2
0,0,a,0
0,0,a,2
3,3,a,0
3,3,a,2
4,4,a,0
4,4,a,2
1,1,b,1


#### 多个DataFrame的简单连接

In [34]:
df1 = DataFrame(np.arange(4), index=['a','b','a','c'])
df2 = DataFrame(np.arange(4), index=['a','b','b','d'])
df3 = DataFrame(np.arange(4), index=['a','b','d','e'])
df4 = DataFrame(np.arange(4), index=['a','b','d','e'])
df1.join([df2,df3,df4], lsuffix=['1','2'], rsuffix=['3','4']) # 多个DataFrame时不起作用了

,0_x,0_y,0_x,0_y
a,0.0,0.0,0.0,0.0
a,2.0,0.0,0.0,0.0
b,1.0,1.0,1.0,1.0
b,1.0,2.0,1.0,1.0
c,3.0,NaN,NaN,NaN
d,NaN,3.0,2.0,2.0
e,NaN,NaN,3.0,3.0


### 轴向连接

另一种数据合并运算也被称作**连接**(concatenation)、**绑定**(binding)或**堆叠**(stacking)；

#### NumPy合并原始数据 -- concatenate

In [37]:
arr = np.arange(16).reshape(4,4)
print np.concatenate([arr,arr], axis=1) # axis为1的合并，也就是横向合并，类似上面的DataFrame的合并
print np.concatenate([arr,arr]) # axis为0的合并，纵向合并，说是堆叠，连接更贴切

[[ 0  1  2  3  0  1  2  3]
 [ 4  5  6  7  4  5  6  7]
 [ 8  9 10 11  8  9 10 11]
 [12 13 14 15 12 13 14 15]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


#### pandas.concat简单连接没有重复索引的三个series，参数全默认

In [39]:
se1 = Series([1,3,5], index=['a','b','c'])
se2 = Series([4,6], index=['d','e'])
se3 = Series([7,8], index=['f','g'])
pd.concat([se1,se2,se3]) # 默认是纵向连接的，也就是堆叠式

a    1
b    3
c    5
d    4
e    6
f    7
g    8
dtype: int64

#### pandas.concat -- 指定参数axis=1，使其横向合并

In [41]:
pd.concat([se1,se2,se3], axis=1) # 能够看到此时出现大量的NaN

,0,1,2
a,1.0,NaN,NaN
b,3.0,NaN,NaN
c,5.0,NaN,NaN
d,NaN,4.0,NaN
e,NaN,6.0,NaN
f,NaN,NaN,7.0
g,NaN,NaN,8.0


#### pandas.concat -- 指定参数join，设置连接方式

In [42]:
pd.concat([se1,se2,se3], join='inner') # 能够看到此时出现大量的NaN

a    1
b    3
c    5
d    4
e    6
f    7
g    8
dtype: int64

#### pandas.concat -- 指定参数join_axes，指定要在其他轴上使用的索引

In [50]:
pd.concat([se1,se2,se3], axis=1, join_axes=[['a','g','e']]) # 注意此处join_axes的写法，是list的list

,0,1,2
a,1.0,NaN,NaN
g,NaN,NaN,8.0
e,NaN,6.0,NaN


#### pandas.concat -- 指定参数keys，创建层次化索引

In [56]:
pd.concat([se1,se1], keys=['first', 'second', 'third'])

first   a    1
        b    3
        c    5
second  a    1
        b    3
        c    5
dtype: int64

In [59]:
# 指定axis为1时，keys作为列名，这个相当于先按axis为0进行concat，然后再unstack转换到DataFrame
pd.concat([se1,se1], axis=1, keys=['first', 'second', 'third'])

,first,second
a,1,1
b,3,3
c,5,5


#### pandas.concat -- 针对DataFrame

In [76]:
df1 = DataFrame(np.arange(16).reshape(4,4), 
                columns=['a','b','c','d'],
               index=['mary','lily','bench','mark'])
df2 = DataFrame(np.arange(16).reshape(4,4), 
                columns=['e','f','g','h'],
               index=['mark','lily','lily','sary'])
print df1
print df2
pd.concat([df1,df2], keys=['level_1','level_2'])

        a   b   c   d
mary    0   1   2   3
lily    4   5   6   7
bench   8   9  10  11
mark   12  13  14  15
       e   f   g   h
mark   0   1   2   3
lily   4   5   6   7
lily   8   9  10  11
sary  12  13  14  15


a     b     c     d     e     f     g     h
level_1 mary    0.0   1.0   2.0   3.0   NaN   NaN   NaN   NaN
        lily    4.0   5.0   6.0   7.0   NaN   NaN   NaN   NaN
        bench   8.0   9.0  10.0  11.0   NaN   NaN   NaN   NaN
        mark   12.0  13.0  14.0  15.0   NaN   NaN   NaN   NaN
level_2 mark    NaN   NaN   NaN   NaN   0.0   1.0   2.0   3.0
        lily    NaN   NaN   NaN   NaN   4.0   5.0   6.0   7.0
        lily    NaN   NaN   NaN   NaN   8.0   9.0  10.0  11.0
        sary    NaN   NaN   NaN   NaN  12.0  13.0  14.0  15.0

#### pandas.concat -- ignore_index忽略跟分析无关的索引

In [80]:
df1 = DataFrame(np.arange(16).reshape(4,4), columns=['a','b','c','d'])
df2 = DataFrame(np.arange(9).reshape(3,3), columns=['a','b','c'])
pd.concat([df1,df2], ignore_index=True) # igonre_index设置为True后，索引会被重新默认值填充掉

,a,b,c,d
0,0,1,2,3.0
1,4,5,6,7.0
2,8,9,10,11.0
3,12,13,14,15.0
4,0,1,2,NaN
5,3,4,5,NaN
6,6,7,8,NaN


#### concat参数

* objs：参与连接的列表或者字典，唯一必填参数；
* axis：连接的轴向，默认为0，纵向；
* join：“inner”交集/“outer”并集，默认是“outer”；
* join_axes：指明用于其他n-1条轴的索引，不执行并集/交集运算；
* keys：与连接对象有关的值，用于形成向上的层次化索引，可以是任意值的列表或数组、元组数组、数组列表（如果将levels设置为多级数组的话）
* levels：指定用作层次化索引各级别上的索引，如果设置了keys的话；
* names：用于创建分层级别的名称，如果设置了keys和levels的话；
* verify_integrity：检查结果对象新轴上的重复情况，发现则报异常，默认为False，即允许重复；
* ignore_index：不保留连接轴上的索引，产生一组新索引range（total_length）；

### 合并重叠数据 -- 用一组数据根据某种条件填充另一组数据

还有一种数据组合问题不能用简单的合并(merge)或连接(concatenation)运算来处理，比如说,你可能有索引全部或部分重叠的两个数据集；

#### 例子：NumPy的where函数,它用于表达一种矢量化的if-else

In [84]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64),
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
print a
print '\n'
print b
print '\n'
# 生成一个布尔型数组用于标示对应位置是从b中获取，还是a，表达如果a中某位置是null，那么该位置的值从b中获取，否则从a中获取
np.where(pd.isnull(a), b, a)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64


f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64




array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

#### Series.combine_first方法 -- 一样实现上述功能，同时还会进行数据对齐

In [91]:
b[:-2].combine_first(a[2:]) # 取b中的0到倒数第2个，以及a中第二个到最后一个，出现重叠优先取b中的生成结果

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

#### DataFrame.combine_first方法

In [98]:
df1 = DataFrame(np.arange(16).reshape(4,4))
df2 = DataFrame(np.arange(9).reshape(3,3))
df1.ix[1:2,1:2] = np.nan
print df1
print '\n'
print df2
print '\n'
df1.combine_first(df2) # 默认使用同样位置的元素去填充df1中的NaN值

    0     1     2   3
0   0   1.0   2.0   3
1   4   NaN   NaN   7
2   8   NaN   NaN  11
3  12  13.0  14.0  15


   0  1  2
0  0  1  2
1  3  4  5
2  6  7  8




/home/helong/anaconda2/envs/python2/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2,3
0,0,1.0,2.0,3
1,4,4.0,5.0,7
2,8,7.0,8.0,11
3,12,13.0,14.0,15


## 重塑和轴向旋转

有许多用于重新排列表格型数据的基础运算。这些函数也称作**重塑**(reshape)或**轴向旋转**(pivot)运算；

### 重塑层次化索引

#### stack -- 将数据的列旋转为行

In [100]:
df = DataFrame(np.arange(4).reshape(2,2),
              columns=['a','b'],
              index=['first','second'])
df.stack() # 原来的columns转换成了内层的索引，DataFrame转成了Series

first   a    0
        b    1
second  a    2
        b    3
dtype: int64

#### unstack -- 将数据的行旋转为列

In [103]:
se = Series(np.arange(4), index=[['a','a','b','b'],['first','second','first','second']])
se.unstack() # 原来的内层索引变成了列名

,first,second
a,0,1
b,2,3


#### 默认unstack、stack都是操作的**最内层**索引，可以通过指定索引号或索引名来指定

In [107]:
se.unstack(0) # 默认总是n-1，指定0操作最外层索引

,a,b
first,0,2
second,1,3


#### 对DataFrame进行stack、unstack时旋转后的轴总是在最内层

In [110]:
df = DataFrame(np.arange(18).reshape(6,3),
              columns=['math','english','chinese'],
              index=[['mary','mary','john','john','lily','lily'],['middle','final','middle','final','middle','final']])

In [112]:
df.unstack() # 将索引转为层次最内层的列，此处使用默认最内层索引

math        english        chinese       
     final middle   final middle   final middle
john     9      6      10      7      11      8
lily    15     12      16     13      17     14
mary     3      0       4      1       5      2

In [113]:
df.unstack().stack(0) # 将原列转为最内层索引

final  middle
john chinese     11       8
     english     10       7
     math         9       6
lily chinese     17      14
     english     16      13
     math        15      12
mary chinese      5       2
     english      4       1
     math         3       0

### 将“长格式”转换为“宽格式” -- 将某个列的全部取值转换成对应的一个一个的列存在，增加列，减少行，所以是长->宽

比如数据库中有一列叫课程，那么就会出现以下这几条记录的情况：
* helong,math,80
* helong,chinese,85
* helong,english,55

这样的设计有好有坏：
* 好处：
    * 课程的取值可以随意扩展；
* 坏处：
    * 数据操作复杂；
    * 数据冗余度高，比如helong由于在三行记录中都有，因此会重复N次，N等于课程的取值数；
    
此处姓名的表示方式就称之为“长格式”，而所谓的转换为“宽格式”如下：

        math,chinese,englisth
    name
        helong,80,85,55

能够看到原来的三行数据变为了一行（长到短），而一行三个数据变为四个（窄变宽）；

#### 将lesson转换到列

In [121]:
df = DataFrame({
    'name':['mary','mary','john','john','lily','lily'],
    'lesson':['chinese','english','chinese','english','chinese','english'],
    'grade':[55,66,77,44,33,88]
})
df

,grade,lesson,name
0,55,chinese,mary
1,66,english,mary
2,77,chinese,john
3,44,english,john
4,33,chinese,lily
5,88,english,lily


In [122]:
df.pivot('name','lesson','grade')

lesson,chinese,english
name,,
john,77,44
lily,33,88
mary,55,66


#### 将lesson转换到列 -- 多数据列

In [123]:
df = DataFrame({
    'name':['mary','mary','john','john','lily','lily'],
    'lesson':['chinese','english','chinese','english','chinese','english'],
    'grade':[55,66,77,44,33,88],
    'xxx':[1,2,3,4,5,6] # 增加的一条数据列，凑成多数据列
})
df

,grade,lesson,name,xxx
0,55,chinese,mary,1
1,66,english,mary,2
2,77,chinese,john,3
3,44,english,john,4
4,33,chinese,lily,5
5,88,english,lily,6


In [125]:
df.pivot('name','lesson') # 多数据列时，忽略第三个参数会生成层次化列，如果指定则只会保留指定的列作为数据列

grade             xxx        
lesson chinese english chinese english
name                                  
john        77      44       3       4
lily        33      88       5       6
mary        55      66       1       2

## 数据转换

## 字符串操作

## 示例：USDA食品数据库